Alterações em relação ao modelo anterior:

- uso de RandomizedSearchCV
- saída de RandomizedSearchCV como entrada para as combinações de GridSearchCV

Com essas modificações, o score pulou de 0.77511 para 0.77990 na competição, posição 5421 (ganho de 726 posições).

<b>Próximos passos</b>

Melhoria dos atributos. Com uso de CV, provavelmente as variáveis derivadas (de SibSp e Parch), que antes não trouxeram melhorias, agora podem trazer resultados, pois, conforme visto anteriormente, elas tem alguma correlação com a variável dependente.

Uma coisa importante, que foi apresentado em algumas soluções, é a remoção da variável PassengerId do modelo de treino; ele não tem correlação com a variável dependente.

Mais um: no modelo inicial que fiz, retirava as variáveis com alto índice de NaN (Cabin, por exemplo); voltar a fazer isso, pois a contribuição dela é nula.

Nos modelos de soluções disponibilizados na Internet, a variável Name permite a criação de derivações conforme o título da pessoa: Mr, Mrs, Miss etc. Verificar se isso pode trazer ganhos.


============================================================================

Foram criadas variáveis derivadas de SibSp e Parch. Não apresentou melhoria no resultado.

As variáveis criadas, como ditas, não apresentaram melhoria. Além disso, conforme apresentado, abaixo, nem todas as variáveis tem a mesma importância.


- Variable: Sex                  Importance: 0.42
- Variable: Fare                 Importance: 0.18
- Variable: Pclass               Importance: 0.12
- Variable: Age                  Importance: 0.11
- Variable: PassengerId          Importance: 0.1
- Variable: no_parch             Importance: 0.02
- Variable: no_sibling           Importance: 0.01
- Variable: one_sibling          Importance: 0.01
- Variable: mult_sibling         Importance: 0.01
- Variable: one_parch            Importance: 0.01
- Variable: mult_parch           Importance: 0.01

Sem a criação de features derivadas, segue a nova importância.

- Variable: Sex                  Importance: 0.41
- Variable: Fare                 Importance: 0.18
- Variable: Pclass               Importance: 0.12
- Variable: Age                  Importance: 0.12
- Variable: PassengerId          Importance: 0.11
- Variable: SibSp                Importance: 0.03
- Variable: Parch                Importance: 0.03


Com isso, vamos manter apenas as variáveis Sex, Fare, Pclass, Age, PassengerId.

Nessa tentativa, vamos usar somente essas features.

PS.: Resultado rodando todas as features e com RandomizedSearchCV

- Variable: Sex                  Importance: 0.39
- Variable: Fare                 Importance: 0.11
- Variable: Ticket               Importance: 0.1
- Variable: Pclass               Importance: 0.09
- Variable: Age                  Importance: 0.09
- Variable: Cabin                Importance: 0.07
- Variable: PassengerId          Importance: 0.06
- Variable: SibSp                Importance: 0.02
- Variable: Parch                Importance: 0.02
- Variable: Embarked             Importance: 0.02
- Variable: Age_na               Importance: 0.01

Conforme suspeita inicial, Embarked não tem importância significativa no resultado. E foi confirmada também a pouca importância de SibSp e Parch. Cabin, que parecia não ter tanta importância, quase que equivale a Age.

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [3]:
PATH = "data/"

In [4]:
!ls {PATH}

gender_submission.csv  preds_07112018_night.csv      test.csv
preds_07102018.csv     preds_07112018_night_new.csv  train.csv
preds_07112018.csv     preds.csv


Vamos ler o arquivo de treino!

In [4]:
df_raw = pd.read_csv(f'{PATH}train.csv')

In [5]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [7]:
df_raw.shape

(891, 12)

In [6]:
df_raw.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [7]:
display_all(df_raw)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.00,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.00,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.00,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.00,1,0,237736,30.0708,NaN,C


In [10]:
len(df_raw[df_raw.Survived == 1])/len(df_raw)

0.3838383838383838

In [8]:
display_all(df_raw[df_raw.Survived == 1])

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.1000,C123,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.00,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.00,1,0,237736,30.0708,NaN,C
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.00,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.00,0,0,113783,26.5500,C103,S
15,16,1,2,"Hewlett, Mrs. (Mary D Kingcome)",female,55.00,0,0,248706,16.0000,NaN,S
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C


Temos que ajustar os dados.

- As colunas 'Name', 'Cabin', 'Ticket' e 'Embarked' (porto de embarcação), em princípio, não fazem sentido. Provavelmente nossa variável dependente não se relacionam com elas.
- A coluna 'Sex', 'Embarked' devem ser categorizada.
- Também temos alguns NaN, que devem ser tratados (vide abaixo).

In [8]:
len(df_raw)

891

In [11]:
(df_raw.isnull().sum()/len(df_raw)).sort_values(ascending=False)

Cabin          0.771044
Age            0.198653
Embarked       0.002245
Fare           0.000000
Ticket         0.000000
Parch          0.000000
SibSp          0.000000
Sex            0.000000
Name           0.000000
Pclass         0.000000
Survived       0.000000
PassengerId    0.000000
dtype: float64

As colunas Age, Cabin e Embarked contém valores inexistentes (<i>missing values</i>).

<i>Nota: a coluna Cabin possui uma alta porcentagem de missing values; além disso, ela não me parece trazer grande informação. Vamos adicioná-la a lista de colunas a serem dropadas.</i>

In [6]:
df_raw.drop(columns=['Name', 'Embarked', 'SibSp', 'Parch'], inplace=True)

In [7]:
df_raw.tail()

,PassengerId,Survived,Pclass,Sex,Age,Ticket,Fare,Cabin
886,887,0,2,male,27.0,211536,13.00,NaN
887,888,1,1,female,19.0,112053,30.00,B42
888,889,0,3,female,NaN,W./C. 6607,23.45,NaN
889,890,1,1,male,26.0,111369,30.00,C148
890,891,0,3,male,32.0,370376,7.75,NaN


In [18]:
m = RandomForestClassifier(n_jobs=-1)
m.fit(df_raw.drop(columns='Survived'), df_raw.Survived)

ValueError: could not convert string to float: 'Q'

Rodei apenas para confirmar que precisamos ajustar dados.

In [8]:
# convertendo para categorias Pandas
train_cats(df_raw)

In [9]:
print(df_raw.Sex.cat.categories)
print(df_raw.Cabin.cat.categories)
print(df_raw.Ticket.cat.categories)

Index(['female', 'male'], dtype='object')
Index(['A10', 'A14', 'A16', 'A19', 'A20', 'A23', 'A24', 'A26', 'A31', 'A32',
       ...
       'E8', 'F E69', 'F G63', 'F G73', 'F2', 'F33', 'F38', 'F4', 'G6', 'T'],
      dtype='object', length=147)
Index(['110152', '110413', '110465', '110564', '110813', '111240', '111320',
       '111361', '111369', '111426',
       ...
       'STON/O2. 3101290', 'SW/PP 751', 'W./C. 14258', 'W./C. 14263',
       'W./C. 6607', 'W./C. 6608', 'W./C. 6609', 'W.E.P. 5734', 'W/C 14208',
       'WE/P 5735'],
      dtype='object', length=681)


In [10]:
df_raw.Sex = df_raw.Sex.cat.codes
df_raw.Cabin = df_raw.Cabin.cat.codes
df_raw.Ticket = df_raw.Ticket.cat.codes

In [11]:
display_all(df_raw)

,PassengerId,Survived,Pclass,Sex,Age,Ticket,Fare,Cabin
0,1,0,3,1,22.00,523,7.2500,-1
1,2,1,1,0,38.00,596,71.2833,81
2,3,1,3,0,26.00,669,7.9250,-1
3,4,1,1,0,35.00,49,53.1000,55
4,5,0,3,1,35.00,472,8.0500,-1
5,6,0,3,1,NaN,275,8.4583,-1
6,7,0,1,1,54.00,85,51.8625,129
7,8,0,3,1,2.00,395,21.0750,-1
8,9,1,3,0,27.00,344,11.1333,-1
9,10,1,2,0,14.00,132,30.0708,-1


In [12]:
df, y, nas = proc_df(df_raw, 'Survived')

In [13]:
display_all(df)

,PassengerId,Pclass,Sex,Age,Ticket,Fare,Cabin,Age_na
0,1,3,1,22.00,523,7.2500,-1,False
1,2,1,0,38.00,596,71.2833,81,False
2,3,3,0,26.00,669,7.9250,-1,False
3,4,1,0,35.00,49,53.1000,55,False
4,5,3,1,35.00,472,8.0500,-1,False
5,6,3,1,28.00,275,8.4583,-1,True
6,7,1,1,54.00,85,51.8625,129,False
7,8,3,1,2.00,395,21.0750,-1,False
8,9,3,0,27.00,344,11.1333,-1,False
9,10,2,0,14.00,132,30.0708,-1,False


In [14]:
# para efeitos de teste, dropar Age_na
df.drop(columns='Age_na', inplace=True)

In [15]:
display_all(df)

,PassengerId,Pclass,Sex,Age,Ticket,Fare,Cabin
0,1,3,1,22.00,523,7.2500,-1
1,2,1,0,38.00,596,71.2833,81
2,3,3,0,26.00,669,7.9250,-1
3,4,1,0,35.00,49,53.1000,55
4,5,3,1,35.00,472,8.0500,-1
5,6,3,1,28.00,275,8.4583,-1
6,7,1,1,54.00,85,51.8625,129
7,8,3,1,2.00,395,21.0750,-1
8,9,3,0,27.00,344,11.1333,-1
9,10,2,0,14.00,132,30.0708,-1


In [16]:
m = RandomForestClassifier(n_jobs=-1)
m.fit(df,y)
m.score(df,y)

0.9910213243546577

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size = 0.25, random_state = 42)
X_train.shape, y_train.shape, X_test.shape

((668, 7), (668,), (223, 7))

In [19]:
#def print_score(m):
def print_score(m, metrics = metrics):
    #res = [m.score(X_train,y_train),m.score(X_test, y_test)]
    res = [metrics.accuracy_score(y_train, m.predict(X_train)), metrics.accuracy_score(y_test, m.predict(X_test))]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [20]:
m = RandomForestClassifier(n_jobs=-1)
m.fit(X_train,y_train)
print_score(m)

[0.9790419161676647, 0.7982062780269058]


Estamos com <b>overfitting</b>; a diferença entre o erro no treinamento e teste é considerável.

Vamos tentar aumentando o número de árvores.

In [21]:
m = RandomForestClassifier(n_jobs=-1, n_estimators=20)
m.fit(X_train,y_train)
print_score(m)

[1.0, 0.8385650224215246]


In [22]:
m = RandomForestClassifier(n_jobs=-1, n_estimators=100, oob_score=True)
m.fit(X_train,y_train)
print_score(m)

[1.0, 0.8251121076233184, 0.812874251497006]


In [23]:
preds = np.stack([t.predict(X_test) for t in m.estimators_])
preds.shape

(100, 223)

In [24]:
m = RandomForestClassifier(n_jobs=-1, n_estimators=1000, oob_score=True)
m.fit(X_train,y_train)
print_score(m)

[1.0, 0.8251121076233184, 0.8173652694610778]


In [25]:
# voltando ao algoritmo inicial, com o número de árvores padrão
m = RandomForestClassifier(n_jobs=-1)
m.fit(X_train,y_train)
print_score(m)

[0.9805389221556886, 0.8340807174887892]


Vamos tentar usar <b>subsampling</b>.

Temos 891 linhas. Subsampling de 100.

In [143]:
len(df)

891

In [26]:
set_rf_samples(100)

In [27]:
m = RandomForestClassifier(n_jobs=-1, oob_score=True)
m.fit(X_train,y_train)
print_score(m)

[0.8248502994011976, 0.7713004484304933, 0.7709580838323353]


O subsampling de 100 não ajudou. Diminuímos o overfitting mas o resultado ainda está ruim.
Vamos combinar com o aumento do número de árvores para ver o que acontece.

In [28]:
m = RandomForestClassifier(n_jobs=-1, n_estimators=1000, oob_score=True)
m.fit(X_train,y_train)
print_score(m)

[0.8937125748502994, 0.8116591928251121, 0.8083832335329342]


In [29]:
# modificar o subsampling
reset_rf_samples()
set_rf_samples(50)

In [30]:
m = RandomForestClassifier(n_jobs=-1, n_estimators=1000, oob_score=True)
m.fit(X_train,y_train)
print_score(m)

[0.8488023952095808, 0.7847533632286996, 0.7979041916167665]


In [31]:
#modificar o subsampling
reset_rf_samples()
set_rf_samples(150)

In [32]:
m = RandomForestClassifier(n_jobs=-1,n_estimators=1000,  oob_score=True)
m.fit(X_train,y_train)
print_score(m)

[0.9176646706586826, 0.8116591928251121, 0.8083832335329342]


In [62]:
for samples in range(30,250,10): # 150 e 200
    reset_rf_samples()
    set_rf_samples(samples)
    m = RandomForestClassifier(n_jobs=-1,n_estimators=1000,  oob_score=True)
    m.fit(X_train,y_train)
    print(samples, end =": ")
    print_score(m)

30: [0.8293413173652695, 0.7713004484304933, 0.7829341317365269]
40: [0.8488023952095808, 0.7937219730941704, 0.7979041916167665]
50: [0.8547904191616766, 0.7757847533632287, 0.7934131736526946]
60: [0.8637724550898204, 0.7937219730941704, 0.8038922155688623]
70: [0.8652694610778443, 0.7892376681614349, 0.812874251497006]
80: [0.8682634730538922, 0.8026905829596412, 0.811377245508982]
90: [0.8712574850299402, 0.7982062780269058, 0.812874251497006]
100: [0.875748502994012, 0.7982062780269058, 0.8143712574850299]
110: [0.875748502994012, 0.8026905829596412, 0.8158682634730539]
120: [0.8772455089820359, 0.8026905829596412, 0.8203592814371258]
130: [0.8817365269461078, 0.8071748878923767, 0.8158682634730539]
140: [0.8907185628742516, 0.8071748878923767, 0.8158682634730539]
150: [0.8922155688622755, 0.8071748878923767, 0.8158682634730539]
160: [0.8922155688622755, 0.8116591928251121, 0.8203592814371258]
170: [0.9011976047904192, 0.8116591928251121, 0.8173652694610778]
180: [0.90269461077844

Com subsampling entre 120 e 130 tivemos um resultado razoável. 

Vamos tentar melhorar com <b>ajuste de parâmetros.</b>

In [33]:
reset_rf_samples()
m = RandomForestClassifier(n_jobs=-1,n_estimators=1000,  oob_score=True)
m.fit(X_train,y_train)
print_score(m)

[1.0, 0.8295964125560538, 0.8158682634730539]


Quero verificar se o conjunto que temos é não balanceado.

In [173]:
df_temp = pd.read_csv(f'{PATH}train.csv')
survived = len(df_temp.loc[df_temp['Survived']==1])
not_survived = len(df_temp.loc[df_temp['Survived']==0])
print(survived/len(df_temp), not_survived/len(df_temp))

0.3838383838383838 0.6161616161616161


O conjunto é de certa forma não balanceado (# classes positivas é consideravelemnte menor do que o de classes negativas).
http://www.chioka.in/class-imbalance-problem/

Nesse caso, conforme apresentado em https://www.analyticsvidhya.com/blog/2016/04/complete-tutorial-tree-based-modeling-scratch-in-python/, é aconselhável escolher um valor baixo para min_samples_leaf.

In [34]:
# agora vamos utilizar o min_samples_leaf
m = RandomForestClassifier(n_estimators=1000, n_jobs=-1, oob_score=True, min_samples_leaf=20)
m.fit(X_train, y_train)
print_score(m)

[0.844311377245509, 0.8026905829596412, 0.7919161676646707]


In [35]:
m = RandomForestClassifier(n_estimators=1000, n_jobs=-1, oob_score=True, min_samples_leaf=10)
m.fit(X_train, y_train)
print_score(m)

[0.875748502994012, 0.820627802690583, 0.8098802395209581]


In [36]:
m = RandomForestClassifier(n_estimators=1000, n_jobs=-1, oob_score=True, min_samples_leaf=5)
m.fit(X_train, y_train)
print_score(m)

[0.9026946107784432, 0.820627802690583, 0.8218562874251497]


In [37]:
m = RandomForestClassifier(n_estimators=1000, n_jobs=-1, oob_score=True, min_samples_leaf=2)
m.fit(X_train, y_train)
print_score(m)

[0.9535928143712575, 0.8295964125560538, 0.8188622754491018]


In [73]:
?RandomForestClassifier

In [38]:
# utilizando o min_sample_leaf sozinho
m = RandomForestClassifier(n_jobs=-1, oob_score=True, min_samples_leaf=5)
m.fit(X_train, y_train)
print_score(m)

/home/andre/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/home/andre/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


[0.8847305389221557, 0.8430493273542601, 0.7904191616766467]


In [39]:
m = RandomForestClassifier(n_estimators=1000, n_jobs=-1, oob_score=True, min_samples_leaf=5)
m.fit(X_train, y_train)
print_score(m)

[0.9011976047904192, 0.820627802690583, 0.8158682634730539]


Com min_samples_leaf=5 <s>min_samples_leaf=10</s>, reduzimos o overfitting e melhoramos um pouco um resultado final.

Vamos combinar com o parâmetro max_features.

Conforme discutido em https://www.analyticsvidhya.com/blog/2016/04/complete-tutorial-tree-based-modeling-scratch-in-python/, sqrt costuma funcionar muito bem. Mas vale a pena tentar também com 30 a 40% das features.

In [40]:
# combinando com max_features
m = RandomForestClassifier(n_estimators=1000, n_jobs=-1, oob_score=True, min_samples_leaf=5, max_features=0.30)
m.fit(X_train, y_train)
print_score(m)

[0.9011976047904192, 0.8251121076233184, 0.8143712574850299]


In [47]:
m = RandomForestClassifier(n_estimators=1000, n_jobs=-1, oob_score=True, min_samples_leaf=5, max_features=0.35)
m.fit(X_train, y_train)
print_score(m)

[0.8932584269662921, 0.8491620111731844, 0.8132022471910112]


In [41]:
m = RandomForestClassifier(n_estimators=1000, n_jobs=-1, oob_score=True, min_samples_leaf=5, max_features=0.40)
m.fit(X_train, y_train)
print_score(m)

[0.9041916167664671, 0.820627802690583, 0.811377245508982]


In [39]:
for mf in range(30,41,1):
    max_features = mf/100
    m = RandomForestClassifier(n_estimators=1000, n_jobs=-1, oob_score=True, min_samples_leaf=5, max_features=max_features)
    m.fit(X_train, y_train)
    print(max_features, end=': ')
    print_score(m)

0.3: [0.8862275449101796, 0.8026905829596412, 0.8218562874251497]
0.31: [0.8832335329341318, 0.7982062780269058, 0.8203592814371258]
0.32: [0.8862275449101796, 0.7982062780269058, 0.8248502994011976]
0.33: [0.8862275449101796, 0.8026905829596412, 0.8308383233532934]
0.34: [0.8832335329341318, 0.8026905829596412, 0.8278443113772455]
0.35: [0.8847305389221557, 0.8116591928251121, 0.8278443113772455]
0.36: [0.8847305389221557, 0.8071748878923767, 0.8233532934131736]
0.37: [0.8862275449101796, 0.8116591928251121, 0.8218562874251497]
0.38: [0.8832335329341318, 0.8026905829596412, 0.8263473053892215]
0.39: [0.8847305389221557, 0.8026905829596412, 0.8203592814371258]
0.4: [0.8892215568862275, 0.8026905829596412, 0.8233532934131736]


<s>Com os valores entre 30 e 40% das features, o score final não foi muito diferente daquele sem a presença do parâmetro.</s>

Os valores de 30 a 40% colocaram uma melhora no resultado.

In [42]:
# combinando com max_features 'sqrt' - esse o valor default do parâmetro
m = RandomForestClassifier(n_estimators=1000, n_jobs=-1, oob_score=True, min_samples_leaf=5, max_features='sqrt')
m.fit(X_train, y_train)
print_score(m)

[0.905688622754491, 0.8161434977578476, 0.8158682634730539]


In [51]:
# combinando com max_features
m = RandomForestClassifier(n_estimators=1000, n_jobs=-1, oob_score=True, min_samples_leaf=5, max_features='log2')
m.fit(X_train, y_train)
print_score(m)

[0.8904494382022472, 0.8491620111731844, 0.8061797752808989]


In [43]:
# combinando com max_features
m = RandomForestClassifier(n_estimators=1000, n_jobs=-1, oob_score=True, min_samples_leaf=5, max_features=0.5)
m.fit(X_train, y_train)
print_score(m)

[0.905688622754491, 0.8295964125560538, 0.8158682634730539]


A melhor combinação, aparentemente, foi com 1000 árvores, min_samples_leaf = <s>10</s> 5  e max_features = 'sqrt'.

Combinar com subsampling para ver se há alguma melhoria.

In [38]:
# 5 a 500, não melhorou; resultado foi pior (385 a 400 resultado se aproximou; com menor overfitting, < oob_score)
# 555 a 600 melhor ( > overfitting e < oob-score)
for samples in range(500,610,10): #600 -> 0.86
    reset_rf_samples()
    set_rf_samples(samples)
    m = RandomForestClassifier(n_estimators=1000, n_jobs=-1, oob_score=True, min_samples_leaf=5, max_features='sqrt')
    m.fit(X_train,y_train)
    print(samples, end =": ")
    print_score(m)

500: [0.8787425149700598, 0.8116591928251121, 0.8083832335329342]
510: [0.8817365269461078, 0.8161434977578476, 0.812874251497006]
520: [0.8772455089820359, 0.8116591928251121, 0.8068862275449101]
530: [0.8802395209580839, 0.8116591928251121, 0.8143712574850299]
540: [0.8787425149700598, 0.820627802690583, 0.812874251497006]
550: [0.8832335329341318, 0.8116591928251121, 0.8098802395209581]
560: [0.8832335329341318, 0.8161434977578476, 0.811377245508982]
570: [0.8817365269461078, 0.820627802690583, 0.811377245508982]
580: [0.8847305389221557, 0.8116591928251121, 0.8173652694610778]
590: [0.8847305389221557, 0.820627802690583, 0.812874251497006]
600: [0.8832335329341318, 0.8161434977578476, 0.812874251497006]


Subsampling de 600 <s>650</s> melhorou um pouco resultado (> overfitting, < accuracy e < oob-score)

Vamos adotar esse modelo e rodar para o set de test.

In [44]:
reset_rf_samples()
set_rf_samples(600)
m = RandomForestClassifier(n_estimators=1000, n_jobs=-1, oob_score=True, min_samples_leaf=5, max_features='sqrt')
m.fit(X_train, y_train)
print_score(m)

[0.9011976047904192, 0.820627802690583, 0.812874251497006]


### Descoberta de features importantes

O trecho abaixo nos permite idendificar as <i>features</i> mais importantes.

In [46]:
feature_list = list(df.columns)
# Get numerical feature importances
importances = list(m.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: Sex                  Importance: 0.39
Variable: Fare                 Importance: 0.11
Variable: Ticket               Importance: 0.1
Variable: Pclass               Importance: 0.09
Variable: Age                  Importance: 0.09
Variable: Cabin                Importance: 0.07
Variable: PassengerId          Importance: 0.06
Variable: SibSp                Importance: 0.02
Variable: Parch                Importance: 0.02
Variable: Embarked             Importance: 0.02
Variable: Age_na               Importance: 0.01


### Melhorando a descoberta de hiperparâmetros

O processo que fizemos até agora - tuning de min_samples_leaf, max_features, n_estimators - é muito manual, pouco produtivo e de certa forma ineficaz. A RandomForest nos fornece diversos parâmetros e descobrir manualmente qual é a melhor combinação entre eles é praticamente impossível - apesar de termos algumas intuições, que usamos até agora.

Sckit learn nos fornece uma maneira melhor de descobrir uma combinação desses parâmetros: RandomizedSearchCV. Essa pesquisa faz uso de CV e escolhe randomicamente os valores dentre os que especificamos no random_grid abaixo.

Vamos utilizar essa ferramenta para tentar descobrir a melhor combinação de hiper parâmetros.

In [47]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 0.3, 0.4]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 5, 10, 20]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 0.3, 0.4], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4, 5, 10, 20], 'bootstrap': [True, False]}


In [50]:
reset_rf_samples()

In [51]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] n_estimators=1200, min_samples_split=5, min_samples_leaf=10, max_features=0.4, max_depth=100, bootstrap=False 
[CV] n_estimators=1200, min_samples_split=5, min_samples_leaf=10, max_features=0.4, max_depth=100, bootstrap=False 
[CV]  n_estimators=1200, min_samples_split=5, min_samples_leaf=10, max_features=0.4, max_depth=100, bootstrap=False, total=   3.4s
[CV] n_estimators=1200, min_samples_split=5, min_samples_leaf=10, max_features=0.4, max_depth=100, bootstrap=False 
[CV]  n_estimators=1200, min_samples_split=5, min_samples_leaf=10, max_features=0.4, max_depth=100, bootstrap=False, total=   3.5s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=10, max_features=auto, max_depth=20, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=10, max_features=auto, max_depth=20, bootstrap=True, total=   0.6s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=10, max_features=aut

[CV] n_estimators=600, min_samples_split=5, min_samples_leaf=2, max_features=0.3, max_depth=None, bootstrap=True 
[CV]  n_estimators=600, min_samples_split=5, min_samples_leaf=2, max_features=0.3, max_depth=None, bootstrap=True, total=   1.4s
[CV] n_estimators=600, min_samples_split=5, min_samples_leaf=2, max_features=0.3, max_depth=None, bootstrap=True 
[CV]  n_estimators=600, min_samples_split=5, min_samples_leaf=2, max_features=0.3, max_depth=None, bootstrap=True, total=   1.3s
[CV] n_estimators=1200, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=30, bootstrap=True 
[CV]  n_estimators=600, min_samples_split=5, min_samples_leaf=2, max_features=0.3, max_depth=None, bootstrap=True, total=   1.4s
[CV] n_estimators=1200, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=30, bootstrap=True 
[CV]  n_estimators=1200, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=30, bootstrap=True, total=   2.5s
[CV] n_estimators=1200, min_

[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   38.2s


[CV]  n_estimators=1200, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=30, bootstrap=True, total=   2.5s
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=20, bootstrap=True 
[CV]  n_estimators=1200, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=30, bootstrap=True, total=   2.5s
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=20, bootstrap=True 
[CV]  n_estimators=2000, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=20, bootstrap=True, total=   4.2s
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=20, bootstrap=True 
[CV]  n_estimators=2000, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=20, bootstrap=True, total=   4.4s
[CV] n_estimators=800, min_samples_split=2, min_samples_leaf=5, max_features=sqrt, max_depth=40, bootstrap=True 
[CV]  n_estimators=800, min_s

[CV]  n_estimators=1600, min_samples_split=2, min_samples_leaf=1, max_features=0.4, max_depth=20, bootstrap=True, total=   3.7s
[CV] n_estimators=1600, min_samples_split=2, min_samples_leaf=1, max_features=0.4, max_depth=20, bootstrap=True 
[CV]  n_estimators=2000, min_samples_split=2, min_samples_leaf=5, max_features=0.3, max_depth=30, bootstrap=True, total=   4.2s
[CV] n_estimators=1600, min_samples_split=2, min_samples_leaf=1, max_features=0.4, max_depth=20, bootstrap=True 
[CV]  n_estimators=1600, min_samples_split=2, min_samples_leaf=1, max_features=0.4, max_depth=20, bootstrap=True, total=   3.9s
[CV] n_estimators=1000, min_samples_split=10, min_samples_leaf=2, max_features=0.4, max_depth=40, bootstrap=False 
[CV]  n_estimators=1600, min_samples_split=2, min_samples_leaf=1, max_features=0.4, max_depth=20, bootstrap=True, total=   3.9s
[CV] n_estimators=1000, min_samples_split=10, min_samples_leaf=2, max_features=0.4, max_depth=40, bootstrap=False 
[CV]  n_estimators=1000, min_sam

...

[CV]  n_estimators=600, min_samples_split=2, min_samples_leaf=10, max_features=sqrt, max_depth=100, bootstrap=False, total=   1.9s
[CV] n_estimators=1800, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=80, bootstrap=True 
[CV]  n_estimators=600, min_samples_split=2, min_samples_leaf=10, max_features=sqrt, max_depth=100, bootstrap=False, total=   2.0s
[CV] n_estimators=1800, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=80, bootstrap=True 
[CV]  n_estimators=1800, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=80, bootstrap=True, total=   5.1s
[CV] n_estimators=1800, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=80, bootstrap=True 
[CV]  n_estimators=1800, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=80, bootstrap=True, total=   4.3s
[CV] n_estimators=1400, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=100, bootstrap=True 
[CV]  n_estimators=1400

[CV]  n_estimators=1200, min_samples_split=10, min_samples_leaf=10, max_features=auto, max_depth=100, bootstrap=False, total=   3.3s
[CV] n_estimators=1000, min_samples_split=10, min_samples_leaf=5, max_features=sqrt, max_depth=40, bootstrap=True 
[CV]  n_estimators=1000, min_samples_split=10, min_samples_leaf=5, max_features=sqrt, max_depth=40, bootstrap=True, total=   2.2s
[CV] n_estimators=1000, min_samples_split=10, min_samples_leaf=5, max_features=sqrt, max_depth=40, bootstrap=True 
[CV]  n_estimators=1000, min_samples_split=10, min_samples_leaf=5, max_features=sqrt, max_depth=40, bootstrap=True, total=   2.4s
[CV] n_estimators=1600, min_samples_split=5, min_samples_leaf=10, max_features=sqrt, max_depth=50, bootstrap=False 
[CV]  n_estimators=1000, min_samples_split=10, min_samples_leaf=5, max_features=sqrt, max_depth=40, bootstrap=True, total=   2.5s
[CV] n_estimators=1600, min_samples_split=5, min_samples_leaf=10, max_features=sqrt, max_depth=50, bootstrap=False 
[CV]  n_estimat

[CV] n_estimators=1200, min_samples_split=10, min_samples_leaf=4, max_features=0.3, max_depth=60, bootstrap=False 
[CV]  n_estimators=1200, min_samples_split=10, min_samples_leaf=4, max_features=0.3, max_depth=60, bootstrap=False, total=   2.6s
[CV] n_estimators=1200, min_samples_split=10, min_samples_leaf=4, max_features=0.3, max_depth=60, bootstrap=False 
[CV]  n_estimators=1200, min_samples_split=10, min_samples_leaf=4, max_features=0.3, max_depth=60, bootstrap=False, total=   2.6s
[CV] n_estimators=1400, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=1200, min_samples_split=10, min_samples_leaf=4, max_features=0.3, max_depth=60, bootstrap=False, total=   2.8s
[CV] n_estimators=1400, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=1400, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True, total=   3.0s
[CV] n_estimators=14

[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.8min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 0.3, 0.4], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4, 5, 10, 20], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [52]:
# vamos ver os melhores parâmetros retornados pela pesquisa randomica
rf_random.best_params_

{'n_estimators': 200,
 'min_samples_split': 10,
 'min_samples_leaf': 5,
 'max_features': 'sqrt',
 'max_depth': None,
 'bootstrap': False}

In [53]:
# Vamos usar esses valores
m = RandomForestClassifier(n_estimators=200, n_jobs=-1, min_samples_split=10, min_samples_leaf=5, max_features='sqrt', max_depth=None, bootstrap=False)
m.fit(X_train, y_train)
print_score(m)

[0.9251497005988024, 0.8340807174887892]


### GridSearch com Cross Validation

O passo anterior nos permitiu descobrir os melhores valores dos parâmetros de forma randômica. Vamos usar agora uma técnica que faz a combinação dos melhores valores que informamos, utilizando todas as combinações possíveis.

In [71]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True, False],
    'max_depth': [80, 90, 100, 110],
    'max_features': ['sqrt'],
    'min_samples_leaf': [2, 5, 10, 20],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [50, 100, 200, 500, 1000]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

In [72]:
# Fit the grid search to the data: 2*4*1*4*3*5 = 480 combinações!
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 480 candidates, totalling 1440 fits
[CV] bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=8, n_estimators=50 
[CV] bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=8, n_estimators=50 
[CV]  bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=8, n_estimators=50, total=   0.2s
[CV] bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=8, n_estimators=50 
[CV]  bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=8, n_estimators=50, total=   0.2s
[CV] bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=8, n_estimators=50, total=   0.2s
[CV] bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=8, n_estimator

[CV]  bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=12, n_estimators=100, total=   0.3s
[CV] bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=12, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1000, total=   3.2s
[CV] bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=12, n_estimators=200 
[CV]  bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=12, n_estimators=100, total=   0.3s
[CV] bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=12, n_estimators=200 
[CV]  bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=12, n_estimators=200, total=   0.5s
[CV] bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=12, n_estimators=200 


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   19.7s


[CV]  bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=12, n_estimators=200, total=   0.6s
[CV] bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=12, n_estimators=500 
[CV]  bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=12, n_estimators=200, total=   0.8s
[CV] bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=12, n_estimators=500 
[CV]  bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=12, n_estimators=500, total=   1.2s
[CV] bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=12, n_estimators=500 
[CV]  bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=12, n_estimators=500, total=   1.3s
[CV] bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=12, n_estimators=1000 
[CV]  bootstrap=True, max_d

[CV]  bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=500, total=   1.4s
[CV] bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=1000 
[CV]  bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=1000, total=   2.4s
[CV] bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=1000 
[CV]  bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=1000, total=   2.2s
[CV] bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=5, min_samples_split=12, n_estimators=50 
[CV]  bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=5, min_samples_split=12, n_estimators=50, total=   0.1s
[CV] bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=5, min_samples_split=12, n_estimators=50 
[CV]  bootstrap=True, max_d

...

[CV] bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=5, min_samples_split=12, n_estimators=500 
[CV]  bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=5, min_samples_split=12, n_estimators=500, total=   1.8s
[CV] bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=5, min_samples_split=12, n_estimators=500 
[CV]  bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=5, min_samples_split=12, n_estimators=500, total=   1.8s
[CV] bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=5, min_samples_split=12, n_estimators=1000 
[CV]  bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=5, min_samples_split=12, n_estimators=500, total=   1.7s
[CV] bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=5, min_samples_split=12, n_estimators=1000 
[CV]  bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=5, min_samples_split=12, n_estimators=1000, total=   3.2s
[CV] boot

[CV]  bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=10, min_samples_split=10, n_estimators=1000, total=   2.1s
[CV] bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=10, min_samples_split=10, n_estimators=1000 
[CV]  bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=10, min_samples_split=10, n_estimators=1000, total=   2.2s
[CV] bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=10, min_samples_split=12, n_estimators=50 
[CV]  bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=10, min_samples_split=12, n_estimators=50, total=   0.2s
[CV] bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=10, min_samples_split=12, n_estimators=50 
[CV]  bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=10, min_samples_split=12, n_estimators=50, total=   0.1s
[CV] bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=10, min_samples_split=12, n_estimators=50 
[CV]  

[CV] bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=20, min_samples_split=10, n_estimators=50 
[CV]  bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=20, min_samples_split=10, n_estimators=50, total=   0.2s
[CV] bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=20, min_samples_split=10, n_estimators=100 
[CV]  bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=20, min_samples_split=10, n_estimators=100, total=   0.3s
[CV] bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=20, min_samples_split=10, n_estimators=100 
[CV]  bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=20, min_samples_split=10, n_estimators=100, total=   0.4s
[CV] bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=20, min_samples_split=10, n_estimators=100 
[CV]  bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=20, min_samples_split=8, n_estimators=1000, total=   2.8s
[CV] b

[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed: 11.6min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'bootstrap': [True, False], 'max_depth': [80, 90, 100, 110], 'max_features': ['sqrt'], 'min_samples_leaf': [2, 5, 10, 20], 'min_samples_split': [8, 10, 12], 'n_estimators': [50, 100, 200, 500, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [73]:
grid_search.best_params_

{'bootstrap': False,
 'max_depth': 90,
 'max_features': 'sqrt',
 'min_samples_leaf': 5,
 'min_samples_split': 8,
 'n_estimators': 200}

In [74]:
# Vamos usar esses valores
m = RandomForestClassifier(n_estimators=200, n_jobs=-1, min_samples_split=8, min_samples_leaf=5, max_features='sqrt', max_depth=90, bootstrap=False)
m.fit(X_train, y_train)
print_score(m)

[0.9251497005988024, 0.8295964125560538]


In [75]:
df_test = pd.read_csv(f'{PATH}test.csv')

In [76]:
len(df_test)

418

In [64]:
display_all(df_test)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.50,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.00,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.00,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.00,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.00,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.00,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.00,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.00,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.00,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.00,2,0,A/4 48871,24.1500,NaN,S


In [77]:
df_test = df_test.assign(Survived=np.nan) # faço isso para usar proc_df

In [78]:
df_test.drop(columns=['Name', 'Embarked', 'SibSp', 'Parch'], inplace=True)
train_cats(df_test)

In [79]:
df_test.Sex = df_test.Sex.cat.codes
df_test.Cabin = df_test.Cabin.cat.codes
df_test.Ticket = df_test.Ticket.cat.codes

In [80]:
df, y, nas = proc_df(df_test, 'Survived')

In [81]:
display_all(df)

,PassengerId,Pclass,Sex,Age,Ticket,Fare,Cabin,Age_na,Fare_na
0,892,3,1,34.50,152,7.8292,-1,False,False
1,893,3,0,47.00,221,7.0000,-1,False,False
2,894,2,1,62.00,73,9.6875,-1,False,False
3,895,3,1,27.00,147,8.6625,-1,False,False
4,896,3,0,22.00,138,12.2875,-1,False,False
5,897,3,1,14.00,261,9.2250,-1,False,False
6,898,3,0,30.00,158,7.6292,-1,False,False
7,899,2,1,26.00,84,29.0000,-1,False,False
8,900,3,0,18.00,100,7.2292,-1,False,False
9,901,3,1,21.00,269,24.1500,-1,False,False


In [82]:
df.drop(columns=['Age_na','Fare_na'], inplace=True)

In [83]:
display_all(df)

,PassengerId,Pclass,Sex,Age,Ticket,Fare,Cabin
0,892,3,1,34.50,152,7.8292,-1
1,893,3,0,47.00,221,7.0000,-1
2,894,2,1,62.00,73,9.6875,-1
3,895,3,1,27.00,147,8.6625,-1
4,896,3,0,22.00,138,12.2875,-1
5,897,3,1,14.00,261,9.2250,-1
6,898,3,0,30.00,158,7.6292,-1
7,899,2,1,26.00,84,29.0000,-1
8,900,3,0,18.00,100,7.2292,-1
9,901,3,1,21.00,269,24.1500,-1


In [84]:
predictions = m.predict(df)

In [85]:
type(predictions)

numpy.ndarray

In [86]:
df['Survived'] = pd.Series(predictions)
display_all(df)

,PassengerId,Pclass,Sex,Age,Ticket,Fare,Cabin,Survived
0,892,3,1,34.50,152,7.8292,-1,0
1,893,3,0,47.00,221,7.0000,-1,1
2,894,2,1,62.00,73,9.6875,-1,0
3,895,3,1,27.00,147,8.6625,-1,0
4,896,3,0,22.00,138,12.2875,-1,1
5,897,3,1,14.00,261,9.2250,-1,0
6,898,3,0,30.00,158,7.6292,-1,1
7,899,2,1,26.00,84,29.0000,-1,0
8,900,3,0,18.00,100,7.2292,-1,1
9,901,3,1,21.00,269,24.1500,-1,0


In [87]:
df_ouput = df.filter(['PassengerId','Survived'], axis = 1)

In [88]:
df_ouput.to_csv(f'{PATH}preds_07122018_gridsearch_cv.csv', index=False)

In [68]:
len(df_ouput)

418